Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [1]:
let name = "Vimarsh Sathia"
let rollno = "CS17B046"

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



val name : string = "Vimarsh Sathia"


val rollno : string = "CS17B046"


## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Mutability and Modules

In this assignment, you will design and implement a couple of mutable data structures and operations on them.

## Problem 1

Implement structures `IntShowable` and `FloatShowable` that satisfy the following signature. Use `string_of_int: int -> string` and `string_of_float: float -> string` for the corresponding `string_of_t` functions.

In [2]:
module type Showable = sig
  type t
  val string_of_t : t -> string
end

module type Showable = sig type t val string_of_t : t -> string end


In [3]:
(* Implement modules IntShowable and FloatShowable *)

module  IntShowable: Showable with type t=int = struct
    type t=int
    let string_of_t u = string_of_int u
end

module FloatShowable: Showable with type t = float = struct
    type t=float
    let string_of_t u = string_of_float u
end

module IntShowable : sig type t = int val string_of_t : t -> string end


module FloatShowable : sig type t = float val string_of_t : t -> string end


In [4]:
assert (IntShowable.string_of_t 10 = "10");
assert (FloatShowable.string_of_t 0.0 = "0.")

- : unit = ()


## Problem 2

Implement a functor 

```ocaml
module MakeNode : 
  functor (C : Showable) -> 
    DoublyLinkedListNode with type content = C.t
``` 

where `DoublyLinkedListNode` is the module type:

In [5]:
module type DoublyLinkedListNode = sig
  type t
  (** The type of doubly linked list node *)
  
  type content
  (** The type of content stored in the doubly linked list node *)

  val create : content -> t 
  (** create a new doubly linked list node with [content] as the content. 
      The next and previous nodes are [None]. *)
  
  val get_next : t -> t option
  (** [get_next t] returns [Some t'] if [t'] is the successor node of [t]. 
      If [t] has no successor, then return [None] *)
      
  val get_prev : t -> t option
  (** [get_prev t] returns [Some t'] if [t'] is the predecessor node of [t]. 
      If [t] has no predecessor, then return [None] *)
      
  val get_content : t -> content
  (** [content t] returns the content [c] of node [t] *)
      
  val set_next : t -> t option -> unit
  (** [set_next t t'] updates the next node of [t] to be [t'] *)
  
  val set_prev : t -> t option -> unit
  (** [set_prev t t'] updates the prev node of [t] to be [t'] *)
  
  val string_of_content : content -> string
  (** [string_of_content c] returns the string form of content [c] *)
end

module type DoublyLinkedListNode =
  sig
    type t
    type content
    val create : content -> t
    val get_next : t -> t option
    val get_prev : t -> t option
    val get_content : t -> content
    val set_next : t -> t option -> unit
    val set_prev : t -> t option -> unit
    val string_of_content : content -> string
  end


In [6]:
(* Implement the functor MakeNode *)
module MakeNode (C : Showable) : DoublyLinkedListNode with type content = C.t = struct
        
    type content = C.t
    
    type t = 
    {
        data : C.t;
        mutable next : t option;
        mutable prev : t option
    }
    
    let create v =  {data = v; next = None; prev = None};;
    
    let get_next ptr = ptr.next;;
    
    let get_prev ptr = ptr.prev;;
    
    let get_content ptr = ptr.data;;
    
    let set_next ptr ele = ptr.next <- ele;;
    
    let set_prev ptr ele = ptr.prev <- ele;;
    
    let string_of_content v  = (C.string_of_t v);;
    
end

module MakeNode :
  functor (C : Showable) ->
    sig
      type t
      type content = C.t
      val create : content -> t
      val get_next : t -> t option
      val get_prev : t -> t option
      val get_content : t -> content
      val set_next : t -> t option -> unit
      val set_prev : t -> t option -> unit
      val string_of_content : content -> string
    end


In [7]:
module IntNode = MakeNode(IntShowable);;

let open IntNode in 
let i1 = create 1 in
assert (get_content i1 = 1)

module IntNode :
  sig
    type t = MakeNode(IntShowable).t
    type content = IntShowable.t
    val create : content -> t
    val get_next : t -> t option
    val get_prev : t -> t option
    val get_content : t -> content
    val set_next : t -> t option -> unit
    val set_prev : t -> t option -> unit
    val string_of_content : content -> string
  end


- : unit = ()


In [8]:
let open IntNode in 
let i1,i2,i3 = create 1, create 2, create 3 in
set_next i1 @@ Some i2;
set_next i2 @@ Some i3;
set_prev i2 @@ Some i1;
set_prev i3 @@ Some i2;
let _ = assert (match get_next i1 with None -> false | Some i2 -> get_content i2 = 2) in
()

- : unit = ()


## Problem 3

Implement a functor 

```ocaml 
module MakeList : 
  functor (N : DoublyLinkedListNode) -> 
    DoublyLinkedList with type node = N.t 
                      and type content = N.content
```

where the module type `DoublyLinkedList` is defined as follows:


In [9]:
module type DoublyLinkedList = sig
  type t 
  (** The type of doubly linked list *)
  
  type node
  (** The type of a doubly linked list node *)
  
  type content
  (** The type of content stored in the [node] of doubly linked list *)
    
  val create : unit -> t 
  (** Creates a new doubly linked list *)
  
  val assign : t -> node option -> unit
  (** [assign t n] makes the node as the head node of the list. 
      The original contents of the list are dropped. *)
  
  val t_of_list : content list -> t
  (** [t_of_list l] returns a new doubly linked list with the list of 
      elements from the list [l]. *)
  
  val is_empty : t -> bool
  (** [is_empty l] return true if [t] is empty *)
  
  val first : t -> node option
  (** [first l] return [Some n] if the list is non-empty. Otherwise, return [None] *)
  
  val insert_first : t -> content -> node
  (** [insert_first l c] inserts a new doubly linked list node [n] as the first 
      node in [l] with [c] as the content. Returns [n]. *)
      
  val insert_after : node -> content -> node
  (** [insert_after n c] inserts a new node [n'] with content [c] after the node [n].
      Returns [n']. *)
  
  val remove : t -> node -> unit
  (** [remove l n] removes the node [n] from the list [l] *)
  
  val iter : t -> (content -> unit) -> unit
  (** [iter l f] applies [f] to each element of the list in the list order. *)
  
  val iter_node : t -> (node -> unit) -> unit
  (** [iter_node l f] applies [f] to each node of the list in the list order. 
      [f] may delete the current node or insert a new node after the current node. 
      In either case, [iter_node] is applied to the rest of the original list. *)
      
  val string_of_t : t -> string
  (** [string_of_t (t_of_list [1;2;3])] returns the string "[1->2->3]".
      [string_of_t (t_of_list [])] returns the string "[]" *)
end

module type DoublyLinkedList =
  sig
    type t
    type node
    type content
    val create : unit -> t
    val assign : t -> node option -> unit
    val t_of_list : content list -> t
    val is_empty : t -> bool
    val first : t -> node option
    val insert_first : t -> content -> node
    val insert_after : node -> content -> node
    val remove : t -> node -> unit
    val iter : t -> (content -> unit) -> unit
    val iter_node : t -> (node -> unit) -> unit
    val string_of_t : t -> string
  end


In [10]:
(* Implement the functor MakeList *)


module MakeList (N : DoublyLinkedListNode) : DoublyLinkedList with type content = N.content 
                                                               and type node = N.t = 
struct
  (* YOUR CODE HERE *)
   type content = N.content
   type node = N.t
   type t = node option ref
   
   
   
   let create () = ref None ;;
   
   let assign ll ptr = ll := ptr;;
   
   let t_of_list vals = 
     let f x acc = 
       let ptr = N.create x in 
         N.set_next ptr !acc; 
         begin match !acc with 
         |None -> ()
         |Some ptr2 -> N.set_prev ptr2 (Some ptr)
         end;
         acc:=Some ptr;
         acc
     in
       List.fold_right f vals (ref None) 
   ;;
     
   let is_empty ll = !ll = None;; 
   
   let first ll = !ll;;
   
   let insert_first ll c = let ptr = N.create c in 
       N.set_next ptr !ll;
       begin match !ll with 
       |None ->()
       |Some ptr2 -> N.set_prev ptr2 (Some ptr)
       end;
       ll:=Some ptr;
       ptr
   ;;
    
   let insert_after ptr1 c = 
     let ptr2 = N.create c in 
       N.set_prev ptr2 (Some ptr1);
       N.set_next ptr2 (N.get_next ptr1);
       N.set_next ptr1 (Some ptr2);
       begin match (N.get_next ptr2) with
       |None -> ()
       |Some ptr3 -> N.set_prev ptr3 (Some ptr2)
       end;
       ptr2
   ;;
       
   let remove ll ptr = 
     let prev = N.get_prev ptr in let next = N.get_next ptr in
     
     begin match prev with
     |Some prev -> N.set_next prev next
     |None -> ll := next
     end;
     
     begin match next with
     |Some next -> N.set_prev next prev
     |None -> ()
     end;
     
     N.set_prev ptr None;
     N.set_next ptr None
   ;;
     
   let iter ll f =
     let rec loop = function
     |None -> ()
     |Some el -> f (N.get_content el); loop (N.get_next el)
     in
       loop !ll
   ;;
   
   let iter_node ll f =
     let rec loop = function
     |None -> ()
     |Some el -> let next_ptr = N.get_next el in f (el); loop (next_ptr)
     in
       loop !ll
   ;;
   
   let string_of_t ll = 
       let rec loop acc l1 = match l1 with
       | None -> acc
       | Some x -> let l2 = (N.get_next x) in 
                   match l2 with
                   |None->loop (acc ^ (N.string_of_content (N.get_content x)) ) (l2)
                   |Some y -> loop (acc ^ (N.string_of_content (N.get_content x)) ^ "->") (l2)
       in
       "[" ^ (loop "" !ll) ^ "]"  
   
end



module MakeList :
  functor (N : DoublyLinkedListNode) ->
    sig
      type t
      type node = N.t
      type content = N.content
      val create : unit -> t
      val assign : t -> node option -> unit
      val t_of_list : content list -> t
      val is_empty : t -> bool
      val first : t -> node option
      val insert_first : t -> content -> node
      val insert_after : node -> content -> node
      val remove : t -> node -> unit
      val iter : t -> (content -> unit) -> unit
      val iter_node : t -> (node -> unit) -> unit
      val string_of_t : t -> string
    end


In [11]:
module L = MakeList(IntNode);;

let open L in
assert (is_empty (t_of_list []));
let l = t_of_list [1;2;3] in
assert (not @@ is_empty l);
let n = match first l with 
| None -> failwith "impossible" 
| Some n -> n
in
assert (1 = IntNode.get_content n);
L.(assign l (first (t_of_list [4;5;6])));
match first l with
| None -> failwith "impossible"
| Some n -> assert (IntNode.get_content n = 4)

module L :
  sig
    type t = MakeList(IntNode).t
    type node = IntNode.t
    type content = IntNode.content
    val create : unit -> t
    val assign : t -> node option -> unit
    val t_of_list : content list -> t
    val is_empty : t -> bool
    val first : t -> node option
    val insert_first : t -> content -> node
    val insert_after : node -> content -> node
    val remove : t -> node -> unit
    val iter : t -> (content -> unit) -> unit
    val iter_node : t -> (node -> unit) -> unit
    val string_of_t : t -> string
  end


- : unit = ()


In [12]:
let open L in
let l = t_of_list [1;2;3] in
let n1 = match first l with
  | Some n1 -> n1
  | None -> failwith "impossible"
in
let n2 = match IntNode.get_next n1 with
  | Some n2 -> n2
  | None -> failwith "impossible"
in
remove l n2;
let n3 = match IntNode.get_next n1 with
  | Some n3 -> n3
  | None -> failwith "impossible"
in
let _ = assert (IntNode.get_content n3 = 3) in
remove l n1;
match first l with
| None -> failwith "impossible"
| Some n3 -> assert (IntNode.get_content n3 = 3)


- : unit = ()


In [13]:
let open L in
let l = t_of_list [1;2;3] in
let s = ref 0 in
iter l (fun c -> s := !s + c);
assert (!s = 6);
iter_node l (fun n -> if IntNode.get_content n mod 2 == 0 then remove l n);
let n1 = match first l with
  | None -> failwith "impossible"
  | Some n -> n
in
assert (IntNode.get_content n1 = 1)


- : unit = ()


## Problem 4

Implement the functor

```ocaml
module MakeDllFunctions :
  functor (N : DoublyLinkedListNode) ->
    Dll_functions with type node = N.t 
                   and type content = N.content 
                   and type dll = MakeList(N).t
```

where the module type `Dll_functions` is:

In [14]:
module type Dll_functions = sig
  type dll
  (** The type of doubly linked list *)
  
  type node
  (** The type of doubly linked list node *)
  
  type content
  (** The type of doubly linked list content *)

  val length : dll -> int
  (** Returns the length of the list. 
      Use [DLL.iter] to implement it. *)
  
  val duplicate : dll -> unit
  (** Given a doubly linked list [l] = [1->2], [duplicate l] returns [1->1->2->2]. 
      Use [DLL.iter_node] to implement it. *)
        
  val rotate : dll -> int -> unit
  (** [rotate l n] rotates the list by [n] nodes. 
      If the list is [1->2->3->4->5], then [rotate l 0] will not modify the list. 
      [rotate l 2] will modify the list to be [3->4->5->1->2]. 
      Assume that [0 <= n < length l]. *)
  
   val reverse : dll -> unit
   (** [reverse l] reverses the list *)
end

module type Dll_functions =
  sig
    type dll
    type node
    type content
    val length : dll -> int
    val duplicate : dll -> unit
    val rotate : dll -> int -> unit
    val reverse : dll -> unit
  end


In [15]:
module MakeDllFunctions (N : DoublyLinkedListNode) 
  : Dll_functions with type node = N.t 
                   and type content = N.content 
                   and type dll = MakeList(N).t = struct

type node = N.t
type content = N.content
type dll = MakeList(N).t
    
module L = MakeList(N);; 
    
let length l = 
  let s = ref 0 in
    L.iter l (fun x -> (s:= !s + 1)) ;!s
    
let duplicate l = 
  let dupaux ptr = 
    let c = N.get_content ptr in
      let _ =  (L.insert_after ptr c) in () 
  in
  L.iter_node l dupaux
;;
let rotate l n = 
  let rec getlastaux p1 =
    match p1 with
    |None->p1
    |Some e1 -> let p2 = (N.get_next e1) in 
       match p2 with 
       |None-> p1
       |Some e2 -> getlastaux p2
    in     
    let rec rotateaux m fst lst curr l =
      begin match curr with 
      |None -> ()
      |Some new_last -> 
         if m=0 then ()
         else if m=1 then let ptr = (N.get_next new_last) in
           begin match ptr with
           |None->()
           |Some new_first ->
            
              N.set_next new_last None;
              begin match lst with
              |None->()
              |Some old_last-> N.set_next old_last fst 
              end;
              
              begin match fst with
              |None->()
              |Some old_first-> N.set_prev old_first lst
              end;
              
              N.set_prev new_first None;
             L.assign l ptr 
            end;
          else rotateaux (m-1) fst lst (N.get_next new_last) l 
        end;
        
      in
      rotateaux n (L.first l) (getlastaux (L.first l)) (L.first l) l
    ;;
      
    let reverse l = 
      (* reverse the given linked list l *)
      let reverseaux cn = 
        let right = N.get_next cn in 
          let left = N.get_prev cn in 
            N.set_next cn left;
            N.set_prev cn right;
            begin match right with
            |None->L.assign l (Some cn)
            |Some trash-> ()
            end
      in
      L.iter_node l reverseaux
      
end

module MakeDllFunctions :
  functor (N : DoublyLinkedListNode) ->
    sig
      type dll = MakeList(N).t
      type node = N.t
      type content = N.content
      val length : dll -> int
      val duplicate : dll -> unit
      val rotate : dll -> int -> unit
      val reverse : dll -> unit
    end


In [16]:
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
let _ = assert (F.length l = 3) in
F.duplicate l;
let _ = assert (F.length l = 6) in
()

- : unit = ()


In [17]:
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
let _ = assert (F.length l = 3) in
F.duplicate l;
let _ = assert (F.length l = 6) in
let _ = assert ("[1->1->2->2->3->3]" = string_of_t l) in
F.rotate l 3;
let _ = assert ("[2->3->3->1->1->2]" = string_of_t l) in
F.rotate l 0;
let _ = assert ("[2->3->3->1->1->2]" = string_of_t l) in
let l = t_of_list [] in
F.rotate l 0;
let _ = assert ("[]" = string_of_t l) in
()

- : unit = ()


In [18]:
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
F.reverse l;
assert ("[3->2->1]" = string_of_t l)


- : unit = ()


## Hash Table

Implement a hash table using [separate chaining](https://en.wikipedia.org/wiki/Hash_table#Separate_chaining_with_linked_lists). Use an OCaml array for the array of buckets. The documentation of OCaml arrays is found in the [OCaml manual](https://caml.inria.fr/pub/docs/manual-ocaml/libref/Array.html).

Each element in the array is a doubly linked list in order to allow chaining. Use your doubly linked list that you have implemented earlier. The hash table will be implemented as a functor that accepts two module arguments. The first one defines the type of the key, size of the array and the hash function. The signature of that module is:

In [19]:
module type Key = sig
  type t
  (** The type of key *)
  
  val num_buckets : int
  (** The number of buckets i.e, the size of the buckets array *)
  
  val hash : t -> int
  (** Hashes the key to an integer between [0, num_buckets) *)
  
  val string_of_t : t -> string
  (** Returns the string representation of key *)
end

module type Key =
  sig
    type t
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


The second functor argument defines the type of content and its signature is `Showable` that we had defined at the beginning of this assignment. 

The signature of the `Hash_table` module is:

In [20]:
module type Hash_table = sig
  type key
  (** The type of key *)
  
  type content
  (** The type of value *)
  
  type t
  (** The type of the hash table *)
  
  val create : unit -> t
  (** Creates a new hash table *)
  
  val put :  t -> key -> content -> unit
  (** [put h k v] associates the key [k] with value [v] in the hash table [h]. 
      If the binding for [k] already exists, overwrite it. *)

  val get : t -> key -> content option
  (** [get h k] returns [Some v], where [v] is the value associated with the 
      key [k] in the hash table [h]. Returns [None] if the [k] is not bound in [h]. *)
      
  val remove : t -> key -> unit
  (** [remove h k] removes the association for key [k] from the hash table [h] if it exists *)
  
  val length : t -> int
  (** Return the number of key-value pairs in the hash table *)
  
  val string_of_t : t -> string
  (** Returns a string version of the hash table. Can be in any format. Only for debugging. *)
  
  val chain_length : t -> int -> int
  (** [chain_length h bucket] returns the length of the doubly linked list 
      at the index [bucket] in the underlying array. 
      Assume that [0 <= bucket < length(bucket_array)] *)
end

module type Hash_table =
  sig
    type key
    type content
    type t
    val create : unit -> t
    val put : t -> key -> content -> unit
    val get : t -> key -> content option
    val remove : t -> key -> unit
    val length : t -> int
    val string_of_t : t -> string
    val chain_length : t -> int -> int
  end


## Problem 5

Implement the functor 

```ocaml
module Make_hash_table :
  functor (K : Key) (C : Showable) ->
    Hash_table with type key = K.t 
                and type content = C.t
```

In [21]:
(* Implement the functor Make_hash_table *)

module Make_hash_table (K:Key) (C:Showable) :
    Hash_table with type key = K.t
                and type content = C.t = struct
    
    module PairShowable: Showable with type t = K.t*C.t = struct
        type t=K.t*C.t
        let string_of_t (u,v) = "lol"
    end
    
    type key = K.t;;
    type content = C.t;;
    
    module DLNode = MakeNode(PairShowable);;
    module DLList = MakeList(DLNode);;
    module F = MakeDllFunctions (DLNode)
    
    type t = DLList.t array;;
    
    
    
    let create () = Array.init (K.num_buckets) (fun (x:int) -> (DLList.create ()) ) ;;
    
    let remove h k =  
      let addr = K.hash k in
        let line = Array.get h addr in
          let rec getnodeaux k ptr = 
            match ptr with
            |None-> None
            |Some node -> let (key,v) = DLNode.get_content node in
                            if(key=k) then ptr
                            else getnodeaux k (DLNode.get_next node)
          in 
            let trash = getnodeaux k (DLList.first line) in
              begin match trash with
              |None->()
              |Some node-> DLList.remove line node 
              end;
              ()
    ;;
    
    
    let put h k v =
      remove h k ;
      let addr = K.hash k in 
        let line = Array.get h addr in
          let _ = DLList.insert_first line (k,v) in 
            ()
    ;;

    let get h k =
      let addr = K.hash k in
        let line = Array.get h addr in
          let rec getaux k ptr = 
            match ptr with 
            |None-> None
            |Some node -> let (key,v) = DLNode.get_content node in
                            if(key=k)then Some v
                            else getaux k (DLNode.get_next node)
          in 
          getaux k (DLList.first line)
    ;;
    
    let chain_length h bucket = 
      let line = Array.get h bucket in
        F.length line
    ;;
    
    let length h = 
      let rec lengthaux acc i = 
        if(i=K.num_buckets)then acc
        else lengthaux (acc+ (chain_length h i)) (i+1)
      in 
        lengthaux 0 0
    ;;
        
    
    let string_of_t h = "notnow";;
    
end

module Make_hash_table :
  functor (K : Key) (C : Showable) ->
    sig
      type key = K.t
      type content = C.t
      type t
      val create : unit -> t
      val put : t -> key -> content -> unit
      val get : t -> key -> content option
      val remove : t -> key -> unit
      val length : t -> int
      val string_of_t : t -> string
      val chain_length : t -> int -> int
    end


In [22]:
module K : Key with type t = int = struct
  type t = int
  let num_buckets = 64
  let hash k = k mod num_buckets
  let string_of_t = string_of_int
end

module StringShowable : Showable with type t = string = struct
  type t = string
  let string_of_t s = s
end

module H = Make_hash_table(K)(StringShowable)

open H
;;

let h = create () in
put h 0 "zero";
put h 1 "one";
assert (get h 0 = Some "zero");
assert (get h 1 = Some "one");
put h 0 "-zero-";
assert (get h 0 = Some "-zero-")


module K :
  sig
    type t = int
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


module StringShowable : sig type t = string val string_of_t : t -> string end


module H :
  sig
    type key = K.t
    type content = StringShowable.t
    type t = Make_hash_table(K)(StringShowable).t
    val create : unit -> t
    val put : t -> key -> content -> unit
    val get : t -> key -> content option
    val remove : t -> key -> unit
    val length : t -> int
    val string_of_t : t -> string
    val chain_length : t -> int -> int
  end


- : unit = ()


In [23]:
let h = create () in
put h 0 "zero";
put h 1 "one";
assert (get h 0 = Some "zero");
assert (get h 1 = Some "one");
put h 64 "-zero-";
assert (get h 0 = Some "zero");
assert (get h 64 = Some "-zero-");
assert (length h = 3);
assert (chain_length h 0 = 2);
assert (chain_length h 1 = 1)


- : unit = ()
